<a href="https://colab.research.google.com/github/hamhead1005/CS-4800-Class-Project/blob/main/Google_Speech_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade google-cloud-speech

In [ ]:
!pip install ez_setup

In [ ]:
!pip install moviepy

In [ ]:
!pip install imageio-ffmpeg

In [ ]:
pip install pysoundfile

In [ ]:
pip install --upgrade google-cloud-storage

In [ ]:
#upload file from PC to local dir. (if needed)
from google.colab import files
uploaded = files.upload()

In [ ]:
#vars
transcriptFileCount = 0 #Used to give transcript files unique names when uploaded to GCS

#imports
import io
import os
import IPython
import json


# Setup for download from GCS (google cloud storage)
from google.colab import auth
from google.cloud import storage
auth.authenticate_user()

project_id = '[cs4800-project-333101]'
!gcloud config set project {'cs4800-project-333101'}

# Download the file from given Google Cloud Storage bucket.
!gsutil cp gs://cs4800_bucket/interview_sample.mp4 /content/trans.mp4 #(Short MP4 File)
!gsutil cp gs://cs4800_bucket/cs4800-project-333101-d0f5d3441758.json /content/cs4800-project-333101-d0f5d3441758.json #Speech to Text/ Cloud Credential
!gsutil cp gs://cs4800_bucket/Long_Recording.mp3 /content/Long_Recording.mp3

In [8]:
#Create credential object and test access
from google.cloud import speech

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/cs4800-project-333101-d0f5d3441758.json'

client = speech.SpeechClient()

In [10]:
#Convert MP4 to MP3
import moviepy
from moviepy.editor import *

mp4_file = '/content/trans.mp4'
mp3_file = '/content/speech.mp3'

vc = moviepy.editor.VideoFileClip(mp4_file)
ac = vc.audio

ac.write_audiofile(mp3_file)

ac.close()
vc.close()

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2162688/45929032 bytes (4.7%)5259264/45929032 bytes (11.5%)8159232/45929032 bytes (17.8%)11247616/45929032 bytes (24.5%)14098432/45929032 bytes (30.7%)17154048/45929032 bytes (37.3%)20275200/45929032 bytes (44.1%)23552000/45929032 bytes (51.3%)26542080/45929032 bytes (57.8%)29769728/45929032 bytes (64.8%)33046528/45929032 bytes (72.0%)36528128/45929032 bytes (79.5%)

100%|██████████| 292/292 [00:00<00:00, 671.47it/s]

[MoviePy] Done.


In [11]:
#audio file too be transcribed (Short)
media_file_name_mp3 = '/content/speech.mp3'

#load media file for API
with open(media_file_name_mp3, 'rb') as f1:
    byte_data_mp3 = f1.read()
audio_mp3 = speech.RecognitionAudio(content=byte_data_mp3)

#Play audio
IPython.display.Audio(media_file_name_mp3) #used for testing

In [ ]:
#API Config for (Small_MP3) Media Files 
config_mp3 = speech.RecognitionConfig(
    sample_rate_hertz=48000,
    enable_automatic_punctuation=True,
    language_code='en-US'
)

In [ ]:
#(Small_MP3) Transcribe RecognitionAudio Objects
response_standard_mp3 = client.recognize(
    config=config_mp3,
    audio=audio_mp3
)

print(response_standard_mp3)

In [ ]:
#Long File Transcription
media_uri = 'gs://cs4800_bucket/Long_Recording.mp3' #Target File from Google Cloud Service
long_aud_mp3 = speech.RecognitionAudio(uri=media_uri)

config_mp3_enhanced = speech.RecognitionConfig(
  sample_rate_hertz=48000,
  enable_automatic_punctuation=True,
  language_code='en-US',
  use_enhanced= True,
  model='video' #Added Precaution
)

#Translate Operation
operation = client.long_running_recognize(
    config=config_mp3_enhanced,
    audio = long_aud_mp3
)

#Text Response
response_long_mp3 = operation.result(timeout=90)
print(response_long_mp3)

In [13]:
#Save output as JSON locally
x = str(response_long_mp3) #String object of response
text_file = open("transcript.json", "w")
text_file.write(x)
text_file.close()

# Confidence Grading Function

(Not)1-10(confident) Score
- Well, hmm, umm, err
  - 10 - count
- Count Pauses
  - 10 - count
- Length of Reply
  - 10 - count

Avg: 3 options --> Final Confidence Score

In [14]:
#Save Results to Google Cloud Storage
storage_client = storage.Client.from_service_account_json('cs4800-project-333101-d0f5d3441758.json')

filename = 'transcripts/transcript_' + str(transcriptFileCount) + '.json' # Save location + Name of File (includes counter)
bucket = storage_client.get_bucket('cs4800_bucket')                       # Name of Bucket
blob = bucket.blob(filename)                                              # Save Location in Bucket + filename
blob.upload_from_filename('transcript.json')                              # Local File to be saved

transcriptFileCount += 1
print(transcriptFileCount)

1
